In [16]:
# TODO: List the directories in the Fortnite folder && date_list file
import os
# fortnite_directory
all_files = os.listdir("test_videos")
fortnite_files = [files for files in all_files if files.rfind("DVR") >= 0]
print(f"fortnite_files: {fortnite_files}")
with open("test_videos/date_list.txt") as file:
    date_times = [line.strip() for line in file if line]

print(f"date_times: {date_times}")

fortnite_files: ['Fortnite 2022.10.05 - 21.37.30.34.DVR.mp4', 'Fortnite 2022.10.05 - 21.35.31.33.DVR.mp4']
date_times: ['22.10.05 - 21.37.33']


In [18]:
# TODO: Get the date/time of each file && date_list file list
from datetime import datetime
# Convert to date/time
fortnite_date_time_format = "%Y.%m.%d - %H.%M.%S"
fortnite_dates = dict([ (datetime.strptime(
        fortnite_file.replace(".DVR", "").replace("Fortnite ", "").replace(".mp4","")[:-3],
        fortnite_date_time_format), fortnite_file) for fortnite_file in fortnite_files ])
print(f"fortnite_dates: {fortnite_dates}")
# https://www.tutorialsteacher.com/articles/convert-string-to-datetime-in-python
# map times to filenames
# print everything

fortnite_dates: {datetime.datetime(2022, 10, 5, 21, 37, 30): 'Fortnite 2022.10.05 - 21.37.30.34.DVR.mp4', datetime.datetime(2022, 10, 5, 21, 35, 31): 'Fortnite 2022.10.05 - 21.35.31.33.DVR.mp4'}


In [ ]:
# TODO: Get files that have "close" times to date_list file
# for every datetime in date_list:
    # for every file_datetime:
        # find all clips within XXX of datetime
# write all clips found to file
# print the result
# TODO (Bonus) copy the chosen files to the "uploads" sub-directory